In [1]:
import pandas as pd
import numpy as np
import scipy
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.stats.multitest import multipletests
import statsmodels.stats.multicomp as multicomp

In [2]:
data_df = pd.read_excel('../data/42255_2025_1311_MOESM5_ESM.xlsx', sheet_name='2I')

In [5]:
data_df = data_df.set_index('Time (min)')

In [6]:
data_df.columns

Index(['HFD', 'HFD.1', 'HFD.2', 'HFD.3', 'HFD.4', 'HFD.5', 'HFD.6', 'HFD+SAL ',
       'HFD+SAL .1', 'HFD+SAL .2', 'HFD+SAL .3', 'HFD+SAL .4', 'HFD+SAL .5',
       'HFD+SAL .6', 'HFD+SAL .7', 'HFD+SANA', 'HFD+SANA.1', 'HFD+SANA.2',
       'HFD+SANA.3', 'HFD+SANA.4', 'HFD+SANA.5', 'HFD+SANA.6', 'HFD+SANA.7'],
      dtype='object')

In [7]:
hfd = ['HFD', 'HFD.1', 'HFD.2', 'HFD.3', 'HFD.4', 'HFD.5', 'HFD.6']
hfd_sal = ['HFD+SAL ',
       'HFD+SAL .1', 'HFD+SAL .2', 'HFD+SAL .3', 'HFD+SAL .4', 'HFD+SAL .5',
       'HFD+SAL .6', 'HFD+SAL .7']
hfd_sana = ['HFD+SANA', 'HFD+SANA.1', 'HFD+SANA.2',
       'HFD+SANA.3', 'HFD+SANA.4', 'HFD+SANA.5', 'HFD+SANA.6', 'HFD+SANA.7']

In [8]:
def reorder_label(label):
    if label[-2:] == 'ND':
        return 'ND vs ' + label.split('vs')[0].strip()
    else:
        return label

In [11]:
for cat in data_df.index:
    
    print(cat)
    grp1, grp2, grp3 =  data_df[hfd].loc[cat].dropna().values, \
                        data_df[hfd_sal].loc[cat].dropna().values, \
                        data_df[hfd_sana].loc[cat].dropna().values

    collect = [grp1, grp2, grp3]
    print('N samples: ' + str(len(np.concatenate(collect))))

    print('ANOVA:')
    f = scipy.stats.f_oneway(grp1, grp2, grp3)
    print('F(2, ' + str(len(np.concatenate(collect))-3) + ') = ' + str(f.statistic) + ', p = ' + str(f.pvalue))

    #print('Bonferroni:')
    comp = multicomp.MultiComparison(data=np.concatenate(collect), 
                                      groups=np.concatenate([['HFD']*len(grp1), 
                                                             ['HFD+SAL']*len(grp2), 
                                                             ['HFD+SANA']*len(grp3)]))
    tbl, a1, a2 = comp.allpairtest(scipy.stats.ttest_ind, method= "bonf", alpha=0.05)
    bonf = pd.DataFrame(tbl)
    bonf.columns = bonf.loc[0].astype(str)
    bonf = bonf[1:]
    bonf['pval_corr'] = a1[2]
    bonf['g1'] = bonf['group1']
    bonf['g2'] = bonf['group2']
    bonf_df = bonf.copy()
    bonf_df['Bonferroni p-value'] = bonf_df['pval_corr']
    bonf_df['Comparison'] = bonf_df['g1'].astype(str) + ' vs ' + bonf_df['g2'].astype(str)
    bonf_df['Comparison'] = bonf_df['Comparison'].apply(reorder_label)

    #print('Tukey:')
    tukey = scipy.stats.tukey_hsd(*collect).pvalue
    tukey_df = pd.DataFrame({'g1':['HFD', 'HFD', 'HFD+SAL'], 
     'g2':['HFD+SAL', 'HFD+SANA', 'HFD+SANA'],
     'Tukey p-value':[tukey[0][1], tukey[0][2], tukey[1][2]]})
    tukey_df['Comparison'] = tukey_df['g1'].astype(str) + ' vs ' + tukey_df['g2'].astype(str)

    combo_df = pd.merge(bonf_df, tukey_df, on=['Comparison'], how='outer')
    combo_df['Published p-value'] = ''
    combo_df = combo_df[['Comparison', 'Published p-value', 'Bonferroni p-value', 'Tukey p-value']].set_index('Comparison')
    print(combo_df.to_markdown())
    
    for grp in collect:
        print(np.mean(grp))

0.0
N samples: 23
ANOVA:
F(2, 20) = 22.47340432496081, p = 7.668700918878939e-06
| Comparison          | Published p-value   |   Bonferroni p-value |   Tukey p-value |
|:--------------------|:--------------------|---------------------:|----------------:|
| HFD vs HFD+SAL      |                     |          0.0939872   |     0.0256438   |
| HFD vs HFD+SANA     |                     |          1.78268e-05 |     5.12947e-06 |
| HFD+SAL vs HFD+SANA |                     |          0.00173111  |     0.00225027  |
0.6519314857142857
0.44698865000000004
0.1735
10.0
N samples: 23
ANOVA:
F(2, 20) = 5.80907327005915, p = 0.010255005017877423
| Comparison          | Published p-value   |   Bonferroni p-value |   Tukey p-value |
|:--------------------|:--------------------|---------------------:|----------------:|
| HFD vs HFD+SAL      |                     |            0.967143  |       0.439358  |
| HFD vs HFD+SANA     |                     |            0.0220716 |       0.0085799 |
| HFD+SAL 

In [10]:
tbl

group1,group2,stat,pval,pval_corr,reject
HFD,HFD+SAL,2.4127,0.0313,0.094,False
HFD,HFD+SANA,7.3071,0.0,0.0,True
HFD+SAL,HFD+SANA,4.4244,0.0006,0.0017,True


Here is Figure 2I:

![file](../img/pub_fig_2i.png)

The statistical procedure used for this panel is not described in the caption. I'll assume that a one-way ANOVA was performed for each group within 0 min, 10 min and 30 min and that either Bonferroni or Tukey post-hoc procedure was performed. Using the [source data](https://www.nature.com/articles/s42255-025-01311-z#Sec38) for this figure, all of these p-values are incorrect both for Bonferroni post-hoc comparison (i.e., independent t-tests between groups and Bonferroni FWER correction on p-values for each pairwise test) and for Tukey's post-hoc test. See summary stats below.

### 0 min

N samples: 23
ANOVA:
F(2, 20) = 22.5, p = 7.67E-6

| Comparison          | Published p-value   |   Bonferroni p-value |   Tukey p-value |
|:--------------------|:--------------------|---------------------:|----------------:|
| HFD vs HFD+SAL      |             0.08        |          0.0940   |     0.0256   |
| HFD vs HFD+SANA     |               0.0003      |          1.783e-05 |     5.13E-6 |
| HFD+SAL vs HFD+SANA |              0.004       |          0.00173  |     0.00225  |

### 10 min

N samples: 23
ANOVA:
F(2, 20) = 5.81, p = 0.0103

| Comparison          | Published p-value   |   Bonferroni p-value |   Tukey p-value |
|:--------------------|:--------------------|---------------------:|----------------:|
| HFD vs HFD+SAL      |            0.60         |            0.967  |       0.439  |
| HFD vs HFD+SANA     |           0.03          |            0.0221 |       0.00858 |
| HFD+SAL vs HFD+SANA |             0.052        |            0.0182 |       0.100  |

### 30 min

N samples: 23
ANOVA:
F(2, 20) = 2.78, p = 0.0859

| Comparison          | Published p-value   |   Bonferroni p-value |   Tukey p-value |
|:--------------------|:--------------------|---------------------:|----------------:|
| HFD vs HFD+SAL      |              0.59       |             0.919 |        0.430 |
| HFD vs HFD+SANA     |            0.21         |             0.191 |        0.0704 |
| HFD+SAL vs HFD+SANA |               0.13      |             0.162 |        0.506 |

Could the authors clarify? Could the authors also comment on why they exclude normal diet (ND) as an experimental condition in this analysis?

The code for this analysis is available at [github.com/reeserich/cal_et_al_2025](https://github.com/reeserich/cal_et_al_2025).

In [14]:
tbl

group1,group2,stat,pval,pval_corr,reject
HFD,HFD+SAL,1.0651,0.3062,0.9186,False
HFD,HFD+SANA,2.0281,0.0636,0.1907,False
HFD+SAL,HFD+SANA,2.1037,0.054,0.1619,False
